## Build database and test data

In [136]:
num_elements = 10000
test_num_elements = 10000

dim = 512
# embedding_list = get_user_embedding_list(num_elements, dim)
# user_embedding_list = get_user_embedding_list(test_num_elements, dim)
# data = embedding_list
# data_labels = np.arange(len(data))
# np.save('../external/train_data_{}' .format(num_elements), embedding_list) 
# np.save('../external/test_data_{}' .format(test_num_elements), user_embedding_list) 


In [1]:
import puffinn

In [2]:
puffinn.Index

puffinn.Index

## Read data

In [137]:
embedding_list = np.load('../external/train_data_{}.npy' .format(num_elements)) 
user_embedding_list = np.load('../external/test_data_{}.npy' .format(test_num_elements)) 

## Baseline, inner product

In [138]:
dot_total_time = 0.0
dot_result = np.zeros(test_num_elements)
for i, user_embedding in enumerate(user_embedding_list):
    start = time.time()
    result = calculate_similarity(user_embedding, embedding_list)
    max_id = np.argmax(result)
#     max_distance = np.max(result)
    dot_time = time.time()-start
    dot_total_time += dot_time
    if i % 1000 == 0:
        print(f"\r dot processed time: {dot_time}")
    dot_result[i] = max_id

print(f"dot avg. time: {dot_total_time/test_num_elements:.6f}")
# np.save('dot_result_{}' .format(num_elements), dot_result)

 dot processed time: 0.002126932144165039
 dot processed time: 0.002033233642578125
 dot processed time: 0.0020051002502441406
 dot processed time: 0.002146482467651367
 dot processed time: 0.0022573471069335938
 dot processed time: 0.0022735595703125
 dot processed time: 0.0021572113037109375
 dot processed time: 0.002063274383544922
 dot processed time: 0.004833221435546875
 dot processed time: 0.0019550323486328125
dot avg. time: 0.002475


## FALCONN

In [2]:
import falconn

In [3]:
falconn.DistanceFunction(1)

DistanceFunction.NegativeInnerProduct

In [141]:
db_size = num_elements
LSHConstructionParameters = falconn.get_default_parameters(
    num_points=db_size, dimension=512, distance=falconn.DistanceFunction(2))
lsh = falconn.LSHIndex(LSHConstructionParameters)
lsh.setup(embedding_list)
query_ob = lsh.construct_query_object()


In [145]:
query_ob.set_num_probes(2000)
total_time = 0.0
find_result = np.zeros(test_num_elements)
for i, user_embedding in enumerate(user_embedding_list):
    start = time.time()
    max_id = query_ob.find_nearest_neighbor(user_embedding)
    duration = time.time()-start
    total_time += duration
    find_result[i] = max_id

print(f"avg. time: {total_time/test_num_elements:.6f}")

print((find_result == dot_result).sum())

avg. time: 0.002206
7537


In [ ]:

# from ngt import base as ngt
# import random
​
# objects = embedding_list
# query = user_embedding_norm
​
# start_time = time.time()
# index = ngt.Index.create(b"tmp", dim)
# index.insert(objects)
# print 'create model time: {}' .format(time.time()-start_time)
# # You can also insert objects from a file like this.
# # index.insert_from_tsv('list.tsv') 
​
# index.save()
# # You can load saved the index like this.
# # index = ngt.Index(b"tmp")
​
# start_time = time.time()
# result = index.search(query, 1)
# print time.time()-start_time
​
# for i, o in enumerate(result) :
#     print(str(i) + ": " + str(o.id-1) + ", " + str(o.distance))
#     object = index.get_object(o.id)
​
​
import hnswlib
import numpy as np
​
class _hnswlib:
    def __init__(self):
        self.model = None
    def model_init(self, dim, space, data, data_labels, ef, ef_construction = 200, M = 16):
​
        max_elements = len(data)
        # Declaring index
        self.model = hnswlib.Index(space, dim) # possible options are l2, cosine or ip
​
        # Initing index - the maximum number of elements should be known beforehand
        self.model.init_index(max_elements, ef_construction, M)
​
        # Element insertion (can be called several times):
        self.model.add_items(data, data_labels)
​
        # Controlling the recall by setting ef:
        self.model.set_ef(ef) # ef should always be > k
    def query_result(self, query, k):
        labels, distances = self.model.knn_query(query, k)
        return labels[0], distances[0]
    def save_model(self, path):
        self.model.save_index("{}" .format(path))
    def load_model(self, space, dim, path, num_elements):
        self.model = hnswlib.Index(space, dim)
        self.model.load_index(path, max_elements = num_elements)
​
​
​
​
​
correct = 0
error = 0
dot_total_time = 0
hnswlib_total_time = 0
_hnswlib = _hnswlib()
​
# generate data
num_elements = 1000000
test_num_elements = 10000
​
dim = 512
########
# embedding_list = get_user_embedding_list(num_elements, dim)
# user_embedding_list = get_user_embedding_list(test_num_elements, dim)
# data = embedding_list
# data_labels = np.arange(len(data))
​
# # user_embedding = np.random.randn(dim)
# # user_embedding_norm = np.array(user_embedding/np.linalg.norm(user_embedding, axis=0, keepdims=True))
​
​
# np.save('train_data_{}' .format(num_elements), embedding_list) 
# user_embedding_list = np.load('test_data_{}.npy' .format(test_num_elements))
# # np.save('test_data_{}' .format(test_num_elements), user_embedding_list) 
​
# dot_result = np.zeros(test_num_elements)
# for i, user_embedding in enumerate(user_embedding_list):
#     start_time = time.time()
#     result = calculate_similarity(user_embedding, embedding_list)
#     max_id = np.argmax(result)
#     max_distance = np.max(result)
#     dot_time = time.time()-start_time
#     dot_total_time += dot_time
#     print 'dot processed time: {}' .format(dot_time)
#     dot_result[i] = max_id
​
# print 'dot avg. time: ', dot_total_time/test_num_elements
# np.save('dot_result_{}' .format(num_elements), dot_result)
####### 
​
embedding_list = np.load('train_data_{}.npy' .format(num_elements))
user_embedding_list = np.load('test_data_{}.npy' .format(test_num_elements))
dot_result = np.load('dot_result_{}.npy' .format(num_elements))
​
data_labels = np.arange(len(embedding_list))
​
ef = 5000
ef_construction = 200
M = 64
_hnswlib.model_init(dim, 'ip', embedding_list, data_labels, ef, ef_construction, M)
_hnswlib.save_model("{}_{}_{}_{}_model.bin" .format(num_elements, ef, ef_construction, M))
# _hnswlib.load_model('ip', dim,  "{}_model.bin" .format(num_elements), num_elements)
​
for i, user_embedding in enumerate(user_embedding_list):
    start_time = time.time()
    labels, distances = _hnswlib.query_result(user_embedding, 100)
    # print labels, distances
    # print -(distances-1), labels
    hnswlib_time = time.time()-start_time
    hnswlib_total_time += hnswlib_time
    print 'hnswlib processed time: {}' .format(hnswlib_time)
    if dot_result[i] in labels:
        correct += 1
    else:
        error += 1
    # break
​
print 'ef = {}, ef_construction = {}, M = {}' .format(ef, ef_construction, M)
print ' - recall rate: {}' .format(float(correct)/(test_num_elements/100))
print ' - hnswlib avg. time: {}' .format(hnswlib_total_time/test_num_elements)